In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

pd.options.display.float_format = '{:,.2f}'.format

<font color = green>

### Question 1
</font>

#### Energy Data

In [2]:
def rc_energy():
    url1 = "http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls"
    energy = pd.read_excel(url1, header = None, skiprows = 18, skipfooter = 38)
    try:
        energy.head()
    except AttributeError:
        print("Помилка при відкритті файлу Energy")
    except NameError:
        print("Помилка при відкритті файлу Energy")
   

    try:
        energy.drop([0, 1], axis = 1, inplace = True)
    except KeyError:
        pass

    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #Будь-які нечислові значення в стовпцях 'Energy Supply', 'Energy Supply per Capita', '% Renewable' замінити на np.nan
    energy[~energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].applymap(np.isreal)] = np.nan 
   
    energy["Energy Supply"] *= 1000000 
    
    energy["Country"].replace('\d','', regex = True, inplace = True)
    energy["Country"].replace(' \(.*?\)','', regex = True, inplace = True)
    
    energy["Country"].replace({"United States of America": "United States",
                               "Republic of Korea": "South Korea",
                               "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                               "China, Hong Kong Special Administrative Region": "Hong Kong"},
                              inplace = True)
    
    return energy

#### GDP Data

In [3]:
def rc_gdp():
    url2 = "http://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=csv"
    fileurl = urlopen(url2)
    zf = ZipFile(BytesIO(fileurl.read()))
    zf.namelist()
    
    for file in zf.namelist():
        # На випадок, якщо база даних оновиться і кінцівка файлу матиме інший CL, не вказую повну назву файлу
        if file.startswith("API_NY.GDP.MKTP.CD_DS2_en_csv"):
            GDP = pd.read_csv(zf.open(file), skiprows = 4) #ЗАПИТАТИСЯ ПРО "Make sure to skip the header"

    try:
        GDP.head()
    except AttributeError:
        print("Помилка при відкритті файлу GDP")
    except NameError:
        print("Помилка при відкритті файлу GDP")
        
    try:
        GDP.drop(["Unnamed: 62"], axis = 1, inplace = True)
    except KeyError:
        pass
    
    GDP["Country Name"].replace(["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"],
                            ["South Korea", "Iran", "Hong Kong"],
                            inplace = True
    )
    
    return GDP

#### ScimEn Data

In [4]:
def rc_scimen():
    url3 = "https://www.scimagojr.com/countryrank.php?category=2102&out=xls"
    ScimEn = pd.read_excel(url3)
    try:
        ScimEn.head()
    except AttributeError:
        print("Помилка при відкритті файлу SciEn")
    except NameError:
        print("Помилка при відкритті файлу SciEn")
   
    return ScimEn

#### Merging Data

In [5]:
def answer_one():
    energy = rc_energy()
    GDP = rc_gdp()
    ScimEn = rc_scimen()
    
    # Тільки назва країни і період з 2008 по 2017
    GDP.drop(GDP.columns[GDP.columns.get_loc('Country Code'):GDP.columns.get_loc('2008')], axis = 1, inplace = True)
    # Top 15 by rank
    ScimEn = ScimEn.sort_values(by = "Rank").head(15)
    
     # 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
    # 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', 
    #'% Renewable', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'
    
    data = pd.merge(ScimEn, energy, how = "inner", left_on = "Country", right_on = "Country")
    data = pd.merge(data, GDP, how = "inner", left_on = "Country", right_on = "Country Name")
    data = data.drop("Country Name", axis = 1)
    data.set_index("Country", inplace = True)
    
    return data

<font color = green>
    
### Answer
</font>

In [6]:
data = answer_one()
data

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country,,,,,,,,,,,,,,,,,,,,
China,1,167992,167369,1057626,722578,6.30,176,127191000000,93,19.75,"4,598,206,091,384.00","5,109,953,609,257.25","6,100,620,488,867.55","7,572,553,836,875.34","8,560,547,314,679.28","9,607,224,481,532.65","10,482,372,109,961.90","11,064,666,282,625.50","11,190,992,550,229.50","12,237,700,479,375.00"
United States,2,126158,123332,1296212,444998,10.27,278,90838000000,286,11.57,"14,718,582,000,000.00","14,418,739,000,000.00","14,964,372,000,000.00","15,517,926,000,000.00","16,155,255,000,000.00","16,691,517,000,000.00","17,427,609,000,000.00","18,120,714,000,000.00","18,624,475,000,000.00","19,390,604,000,000.00"
Japan,3,37948,37612,316956,85620,8.35,155,18984000000,149,10.23,"5,037,908,465,114.48","5,231,382,674,593.70","5,700,098,114,744.41","6,157,459,594,823.72","6,203,213,121,334.12","5,155,717,056,270.83","4,850,413,536,037.84","4,394,977,752,877.82","4,949,273,341,993.88","4,872,136,945,507.59"
United Kingdom,4,28998,28176,335914,64609,11.58,170,7920000000,124,10.60,"2,890,564,338,235.29","2,382,825,985,355.97","2,441,173,394,729.62","2,619,700,404,733.37","2,662,085,168,498.93","2,739,818,680,930.19","3,022,827,781,881.39","2,885,570,309,160.86","2,650,850,178,102.14","2,622,433,959,604.16"
India,5,24872,24300,215787,68150,8.68,141,33195000000,26,14.97,"1,186,952,757,636.11","1,323,940,295,874.06","1,656,617,073,124.71","1,823,049,927,772.05","1,827,637,859,136.23","1,856,722,121,394.42","2,039,127,446,299.30","2,102,390,808,997.09","2,274,229,710,530.03","2,597,491,162,897.67"
Germany,6,24407,23963,231800,48820,9.50,151,13261000000,165,17.90,"3,752,365,607,148.09","3,418,005,001,389.27","3,417,094,562,648.95","3,757,698,281,117.55","3,543,983,909,148.01","3,752,513,503,278.41","3,890,606,893,346.69","3,375,611,100,742.22","3,477,796,274,496.80","3,677,439,129,776.60"
Russian Federation,7,23361,23196,55495,22485,2.38,69,30709000000,214,17.29,"1,660,844,408,499.61","1,222,643,696,991.85","1,524,916,112,078.87","2,051,661,732,059.78","2,210,256,976,945.38","2,297,128,039,058.21","2,063,662,665,171.89","1,368,400,705,491.02","1,284,727,602,173.71","1,577,524,145,963.17"
Canada,8,22908,22465,332093,62436,14.50,177,10431000000,296,61.95,"1,549,131,208,997.19","1,371,153,004,986.44","1,613,464,422,811.13","1,788,647,906,047.76","1,824,288,757,447.57","1,842,628,005,830.18","1,799,268,695,861.80","1,559,623,393,038.66","1,535,767,736,946.18","1,653,042,795,255.04"
France,9,17569,17230,203486,43210,11.58,139,10597000000,166,17.02,"2,918,382,891,460.38","2,690,222,283,967.77","2,642,609,548,930.36","2,861,408,170,264.61","2,683,825,225,092.63","2,811,077,725,703.59","2,852,165,760,630.27","2,438,207,896,251.84","2,465,134,297,438.91","2,582,501,307,216.42"


<font color = green>
    
### Question 2
</font>

In [7]:
def answer_two():
    datacpy = answer_one() # Так як Завдання 2 не вимагає створення нового стовпця в df, то працюватиму на окремому df
    datacpy["avgGDP"] = datacpy.iloc[:,-10:].mean(axis = 1, skipna = True)
    return datacpy["avgGDP"].sort_values(ascending = False) 

<font color = green>
    
### Answer
</font>

In [8]:
answer_two()

Country
United States        16,602,979,300,000.00
China                 8,652,483,724,478.80
Japan                 5,255,258,060,329.84
Germany               3,606,311,426,309.26
France                2,694,553,510,695.68
United Kingdom        2,691,785,020,123.19
Brazil                2,123,361,572,110.79
Italy                 2,095,933,737,999.86
India                 1,868,815,916,366.17
Russian Federation    1,726,176,608,443.35
Canada                1,653,701,592,722.19
Spain                 1,387,494,804,334.45
Australia             1,298,013,130,296.13
South Korea           1,246,918,232,840.51
Iran                    463,580,754,044.25
Name: avgGDP, dtype: float64

<font color = green>
    
### Question 3
</font>

In [9]:
def answer_three():
    datacpy = answer_one() # Так як Завдання 3 не вимагає створення нового стовпця в df, то працюватиму на окремому df
    datacpy["avgGDP"] = answer_two()
    return abs(
            datacpy.sort_values(by = "avgGDP", ascending = False).iloc[5][-2] -
            datacpy.sort_values(by = "avgGDP", ascending = False).iloc[5][-11]
    )

<font color = green>
    
### Answer
</font>

In [10]:
answer_three()

268130378631.12988

<font color = green>
    
### Question 4
</font>

In [11]:
def answer_four():
    data["Citation Ratio"] = data["Self-citations"] / data["Citations"]
    first = data.sort_values(by = "Citation Ratio", ascending = False).iloc[0]
    return first.name, first["Citation Ratio"]

<font color = green>
    
### Answer
</font>

In [12]:
answer_four()

('China', 0.6832074854438147)

<font color = green>
    
### Question 5
</font>

In [13]:
def answer_five():
    data["Population"] = data["Energy Supply"] / data["Energy Supply per Capita"]
    return data.sort_values(by = "Population", ascending = False).iloc[2].name

<font color = green>
    
### Answer
</font>

In [14]:
answer_five()

'United States'

<font color = green>
    
### Question 6
</font>

In [15]:
def answer_six():
    data["DocsPerPerson"] = data["Citations"] / data["Population"]
    data["DocsPerPerson"] = np.float64(data["DocsPerPerson"])
    data["Energy Supply per Capita"] = np.float64(data["Energy Supply per Capita"])
    return data["DocsPerPerson"].corr(data["Energy Supply per Capita"], method = "pearson")

<font color = green>
    
### Answer
</font>

In [16]:
print(answer_six())

0.6642822217071158


<font color = green>
    
### Question 7
</font>

In [17]:
def answer_seven():
    ContinentDict  = {'China':'Asia',
                      'United States':'North America',
                      'Japan':'Asia',
                      'United Kingdom':'Europe',
                      'Russian Federation':'Europe',
                      'Canada':'North America',
                      'Germany':'Europe',
                      'India':'Asia',
                      'France':'Europe',
                      'South Korea':'Asia',
                      'Italy':'Europe',
                      'Spain':'Europe',
                      'Iran':'Asia',
                      'Australia':'Australia',
                      'Brazil':'South America'
                     }
    data["Continent"] = pd.Series(ContinentDict)
    data["Population"] = np.float64(data["Population"])
    return data.groupby('Continent')["Population"].agg([np.size, np.sum, np.mean, np.std])

<font color = green>
    
### Answer
</font>

In [18]:
answer_seven()

,size,sum,mean,std
Continent,,,,
Asia,5.00,"2,898,666,386.61","579,733,277.32","679,097,888.37"
Australia,1.00,"23,316,017.32","23,316,017.32",nan
Europe,6.00,"457,929,667.22","76,321,611.20","34,647,667.07"
North America,2.00,"352,855,249.48","176,427,624.74","199,669,644.86"
South America,1.00,"205,915,254.24","205,915,254.24",nan
